In [4]:
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from numba import jit
import math

import xgboost as xgb

from extract import extract_url

In [ ]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [57]:
def calculerLag(df,lag):
    df_lag = pd.DataFrame()
    for column in df.columns:
        df_column = pd.DataFrame(df[column])
        for i in range(lag):
            df_column['diff' + str(i+1)] = 0
            df_column['diff' + str(i+1)] = df_column[column].diff(i+1)
        
        df_column['Pages'] = column
        df_column = df_column.drop([column],axis= 1)
        df_lag.append(df_column)
        df_lag = pd.concat([df_lag, df_column], ignore_index=False)
    return(df_lag)

In [58]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:5]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
calculerLag(train_1_reduce,2).reset_index()

,Dates,diff1,diff2,Pages
0,2015-07-01,NaN,NaN,2NE1_zh.wikipedia.org_all-access_spider
1,2015-07-02,-7.0,NaN,2NE1_zh.wikipedia.org_all-access_spider
2,2015-07-03,-6.0,-13.0,2NE1_zh.wikipedia.org_all-access_spider
3,2015-07-04,8.0,2.0,2NE1_zh.wikipedia.org_all-access_spider
4,2015-07-05,1.0,9.0,2NE1_zh.wikipedia.org_all-access_spider
...,...,...,...,...
2745,2016-12-27,8.0,-2.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2746,2016-12-28,16.0,24.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2747,2016-12-29,-14.0,2.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2748,2016-12-30,23.0,9.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...


In [7]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:100]
#train_1_reduce = pd.merge(train_1_reduce, extract_url(train_1_reduce['Page']))
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
#train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
Page,,,,,,,,,,,,,,,,,,,,,
2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,24.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,4.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,16.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [55]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1_reduce = train_1[0:5]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
calculerLag(train_1_reduce,2).reset_index()

            2NE1_zh.wikipedia.org_all-access_spider  diff1  diff2  \
Dates                                                               
2015-07-01                                     18.0    NaN    NaN   
2015-07-02                                     11.0   -7.0    NaN   
2015-07-03                                      5.0   -6.0  -13.0   
2015-07-04                                     13.0    8.0    2.0   
2015-07-05                                     14.0    1.0    9.0   

                                              Pages  
Dates                                                
2015-07-01  2NE1_zh.wikipedia.org_all-access_spider  
2015-07-02  2NE1_zh.wikipedia.org_all-access_spider  
2015-07-03  2NE1_zh.wikipedia.org_all-access_spider  
2015-07-04  2NE1_zh.wikipedia.org_all-access_spider  
2015-07-05  2NE1_zh.wikipedia.org_all-access_spider  
            diff1  diff2                                    Pages
Dates                                                            
2015-0

,Dates,diff1,diff2,Pages
0,2015-07-01,NaN,NaN,2NE1_zh.wikipedia.org_all-access_spider
1,2015-07-02,-7.0,NaN,2NE1_zh.wikipedia.org_all-access_spider
2,2015-07-03,-6.0,-13.0,2NE1_zh.wikipedia.org_all-access_spider
3,2015-07-04,8.0,2.0,2NE1_zh.wikipedia.org_all-access_spider
4,2015-07-05,1.0,9.0,2NE1_zh.wikipedia.org_all-access_spider
...,...,...,...,...
2745,2016-12-27,8.0,-2.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2746,2016-12-28,16.0,24.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2747,2016-12-29,-14.0,2.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...
2748,2016-12-30,23.0,9.0,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...


In [10]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE'])

for column in train_1_reduce.columns:
    df = pd.DataFrame(train_1_reduce[column])
    lag = 8
    for i in range(lag):
        df['diff' + str(i+1)] = 0
        df['diff' + str(i+1)] = df[column].diff(i+1)
    X = df.iloc[:,1:9]
    Y = df.iloc[:,0]
    Xtrain = X[0:500]
    Xtest = X[500:550]
    Ytrain = Y[0:500]
    Ytest = Y[500:550]
    
    '''dtrain = xgb.DMatrix(Xtrain, label=Ytrain)
    dtest = xgb.DMatrix(Xtest, label=Ytest)
    param = {'max_depth':2, 'eta':1, 'objective=reg' :' tweedie'}
    num_round = 2
    bst = xgb.train(param, dtrain, num_round)
    preds = bst.predict(dtest)
    dfPredictions = pd.DataFrame(Ytest)
    dfPredictions['predictions'] = preds
    rmse = np.sqrt(mean_squared_error(dfPredictions[column], dfPredictions['predictions']))
    smape = smape_fast(dfPredictions[column].values,  dfPredictions['predictions'].values)
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    metrics_df.loc[column] = [rmse,smape]'''

In [11]:
Xtrain

,diff1,diff2,diff3,diff4,diff5,diff6,diff7,diff8
Dates,,,,,,,,
2015-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-03,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-04,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2015-07-05,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2016-11-07,-3.0,4.0,-2.0,0.0,-2.0,-20.0,-6.0,3.0
2016-11-08,-3.0,-6.0,1.0,-5.0,-3.0,-5.0,-23.0,-9.0
2016-11-09,3.0,0.0,-3.0,4.0,-2.0,0.0,-2.0,-20.0


In [1]:
metrics_df

NameError: name 'metrics_df' is not defined

In [ ]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()

In [ ]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)